## Running in Vertex Pipelines

To make our model training more reproducible, we would like to run it in an automated pipeline that clearly defines the different steps we need to take (e.g. training and evaluating the model) and captures any produced artifacts (e.g. the trained model). In the Google Cloud, we can use Vertex Pipelines for this purpose. 

Vertex Pipelines allows you to define pipelines as a graph of containerized tasks, in which each task performs a specific step needed to train/evaluate/deploy a model. In this notebook we'll define a pipeline interactively and see how we can use it to run our model on top of Vertex's serverless compute infrastructure.

First, let's do some quick setup to set some variables we'll need later on. 

In [ ]:
# Create a folder for storing our component/pipeline artifacts.
! mkdir -p _artifacts

# Set up some required variables.
GCP_REGION = "europe-west3"

# Enter your name here. We'll use this to tag your unique
# Docker image to avoid clashing with other people.
USER_NAME = input("Your user name:")

Next, to be able to run our model code as a containerized task in our Vertex Pipeline, we need to build it into a Docker image and push it to a registry that Vertex AI can access.

You can use the following command to do so:

In [ ]:
! make -C ../ USER_NAME=$USER_NAME docker-push

**Exercise 1 - Building the Docker image**

* Check out the Makefile to see what this command does.
* Inspect the Dockerfile to see what's inside our Docker image. 

Now we've created our Docker image, we'll start building our pipeline. 

Vertex Pipelines can be defined using the [Kubeflow Pipeline SDK](https://www.kubeflow.org/docs/components/pipelines/sdk/). In this SDK, tasks are typically defined using components, which reference a Docker image and define the operations that will be run in the image. These tasks can be combined using their inputs and outputs to define the overall pipeline.

The easiest way to create a component in Python is using the `@component` decorator, which converts a Python function into a pipeline component:

```
@component(
    base_image="my-image",
    output_component_file="_artifacts/train.yaml",
)
def train(train_data_path: str, model: Output[Model]) -> None:
    ...
```

This effectively tasks the code inside the function (`...`) and ensures the code will be run inside the referenced container when the task is executed. 

The component also takes input parameters allowing you to define inputs/outputs for the component (marked by `Input` or `Output` types) or other extra parameters. Note that adding typing for your parameters is crucial when defining a component, as Kubeflow uses these types when compiling your component.

For more information on building Python-based components see [here](https://www.kubeflow.org/docs/components/pipelines/sdk/python-function-components/). 

**Exercise 2 - Building and running our first pipeline**

Let's build and run our first pipeline. Below we've provided the skeleton of a pipeline that defines a `train` task and an (empty) `evalute` task. 

1. Read through the code and see if you understand what it does.
    * How is the `train` component defined? What does it do?
    * How is the pipeline defined? What steps does it include?
2. Run the cell to make sure the pipeline is defined.

In [ ]:
from typing import Optional, NamedTuple

import kfp
from kfp import components
from kfp.v2 import compiler
from kfp.v2.dsl import (
    component,
    Input,
    InputPath,
    OutputPath,
    Output,
    Dataset,
    Metrics,
    Model
)

@component(
    base_image=f"{GCP_REGION}-docker.pkg.dev/gdd-cb-vertex/docker/fancy-fashion-{USER_NAME}",
    output_component_file="_artifacts/train.yaml",
)
def train(train_data_path: str, model: Output[Model]) -> None:
    """Trains the model on the given dataset."""
    
    from pathlib import Path
    import joblib
    
    from fancy_fashion.model import train_model
    from fancy_fashion.util import local_gcs_path
    
    trained_model = train_model(local_gcs_path(train_data_path))

    model_dir = Path(model.path)
    model_dir.mkdir(parents=True, exist_ok=True)
    joblib.dump(trained_model, model_dir / "model.pkl")

    
@component(
    base_image=f"{GCP_REGION}-docker.pkg.dev/gdd-cb-vertex/docker/fancy-fashion-{USER_NAME}",
    output_component_file="_artifacts/evaluate.yaml",
)
def evaluate(
    test_data_path: str, model: InputPath("Model"), metrics: Output[Metrics]
) -> NamedTuple("EvalModelOutput", [("roc", float)]):
    # TODO: Implement the actual evaluation.
    #       Tip: we can use the evaluate_model function from our package.
    metrics.log_metric("roc", 0.9)

    
@kfp.dsl.pipeline(name="fancy-fashion-julian")
def pipeline(train_path: str):
    train_task = train(train_path)
    
    # TODO: Add an evaluate task that uses the evaluate component above.

3. Now we've defined the pipeline, let's compile it using the code below:

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="_artifacts/pipeline.json",
)

4. Finally, let's submit the compiled pipeline by creating and submitting a Pipeline job:

In [ ]:
from google.cloud.aiplatform.pipeline_jobs import PipelineJob

job = PipelineJob(
    display_name=f"fancy-fashion-{USER_NAME}",
    enable_caching=False,
    template_path="_artifacts/pipeline.json",
    parameter_values={
        "train_path": "gs://gdd-cb-vertex-fashion-inputs/train"
    },
    pipeline_root=f"gs://gdd-cb-vertex-fashion-artifacts/pipelines",
    location=GCP_REGION,
)

job.run(
    service_account=f"vmd-fashion@gdd-cb-vertex.iam.gserviceaccount.com"
)

5. Follow the pipeline run by clicking on the printed link. Does it finish successfully?

**Exercise 3 - Extending the pipeline**

Now we have an idea of how Kubeflow Pipelines are defined, lets see if we can add some extra functionality to the pipeline.

1. Extend the pipeline by implementing the `evaluate` component to call our model evaluation function (defined in `model.py`). 
2. Add an extra step `predict` that generates model predictions using our prediction function. See if you can register the generated predictions as a `Dataset` artifact. (Tip: you can do so by passing an `Output` to your component function).

**Exercise 4 - Validating your predictions**

1. Check the predictions you generated against our validation set using the `validate_predictions` function from `validation.py`.
2. Is your model as accurate as you expected? If not, do you have any idea why?


**Bonus exercises**

Should you finish early, try exploring the following:

* Can you find the pipeline runs and the generated artifacts in the console?
* Where are the artifacts stored in GCP?
* Can you retrieve these metadata details programatically? What about the model artifact path in GCS? (Tip: check the `get_pipeline_df` function in the `google.cloud.aiplatform` package and the `MetadataServiceClient` in the `google.cloud.aiplatform_v1` package).
* How can you tell Kubeflow to use a GPU instance for running a specific task?
* Can you deploy the trained model as a REST API endpoint?